#Network analysis with SGraph

Pavan Akula

June 10, 2017

#Abstract
The goal of this assignment is to create a basic graph, such as the Krackhardt kite using Graphlab Create, Python package.

#Introduction

As part of the assignment, I will be presenting Network Analysis of Facebook, association with other companies. I will demonstrate up to second level analysis.

Facebook(Executives and Board of Directors) -> Related to other companies Board of Directors and so on ...

#Software Usage

I will be using Ananconda and Spyder Python 2.7 version as a development platform. Jupyter notebook for documentation. To create graphs I will be using Graphlab Create. For data persistence, MySQL relational database is used.

#Data Source

Data are harvested from MorningStar, Inc website. MorningStar, Inc is an investment research and investment management firm headquartered in Chicago, Illinois. Web scrapping technique is used to harvest data. More information can found at http://www.morningstar.com/. Stock ticker symbol are obtained using finsymbols package.

#Packages used

finsymbols extract stock ticker symbol
bs4
pandas
requests
time
graphlab
IPython.display

In [ ]:
#Create MySQL schema and tables
#Run the code in MySQL Workbench

"""
CREATE SCHEMA IF NOT EXISTS `company`;
USE `company` ;

CREATE TABLE IF NOT EXISTS `company`.`stockExchange`
(
    stockExchangeId INT AUTO_INCREMENT PRIMARY KEY, 
    company NVARCHAR(200) NULL, 
    symbol VARCHAR(10) NULL,
    sector VARCHAR(200) NULL,
    industry VARCHAR(200) NULL,
    stockExchange VARCHAR(10) NULL
);

CREATE TABLE IF NOT EXISTS `company`.`keyExecutive`
(
    keyExecutiveId INT AUTO_INCREMENT PRIMARY KEY,
    executiveName NVARCHAR(100) NULL,
    executiveTitle NVARCHAR(200) NULL,
    company NVARCHAR(200) NULL, 
    stockSymbol VARCHAR(10) NULL,
    stockExchange VARCHAR(10) NULL
);

CREATE TABLE IF NOT EXISTS `company`.`companyBoard`
(
    companyBoardId INT AUTO_INCREMENT PRIMARY KEY, 
    memberName NVARCHAR(100) NULL,
    memberTitle NVARCHAR(200) NULL,
    company NVARCHAR(200) NULL, 
    stockSymbol VARCHAR(10) NULL,
    stockExchange VARCHAR(10) NULL
);
"""

In [ ]:
#Get stock ticker symbol and save them to MySQL databse.
#Run the code in Spyder python 2.7

from finsymbols import symbols
import mysql.connector

"""
amex = symbols.get_amex_symbols()
nasdaq = symbols.get_nasdaq_symbols()
nyse = symbols.get_nyse_symbols()

#Store stock ticker data into MySQL
cnx = mysql.connector.connect(user='root', host='127.0.0.1', database='company')
cursor = cnx.cursor()
add_stocks = ("INSERT INTO stockExchange "
                "(sector, industry, symbol, company, stockExchange) "
                "VALUES (%(sector)s, %(industry)s, %(symbol)s, %(company)s, %(stockExchange)s)")

for s in amex:
    mData = {}
    mData['sector'] = s['sector']
    mData['industry'] = s['industry']
    mData['symbol'] = s['symbol']
    mData['company'] = s['company']
    mData['stockExchange'] = 'amex'
    cursor.execute(add_stocks, mData)

for s in nasdaq:
    mData = {}
    mData['sector'] = s['sector']
    mData['industry'] = s['industry']
    mData['symbol'] = s['symbol']
    mData['company'] = s['company']
    mData['stockExchange'] = 'nasdaq'
    cursor.execute(add_stocks, mData)

for s in nyse:
    mData = {}
    mData['sector'] = s['sector']
    mData['industry'] = s['industry']
    mData['symbol'] = s['symbol']
    mData['company'] = s['company']
    mData['stockExchange'] = 'nyse'
    cursor.execute(add_stocks, mData)
    
cnx.commit()
cursor.close()
cnx.close()
"""

In [ ]:
#Get company board of directors and executive information and save them to MySQL databse.
#Run the code in Spyder python 2.7

import mysql.connector
from bs4 import BeautifulSoup
import requests
import time

"""
#Morning web page
webpage = 'http://insiders.morningstar.com/insiders/trading/current-insiders-list.action?&t=exchange:ticker&region=usa&culture=en-US&cur=&type=callType'

#Create connection - MySQL
cnx1 = mysql.connector.connect(user='root', host='127.0.0.1', database='company')
cnx2 = mysql.connector.connect(user='root', host='127.0.0.1', database='company')

cursor1 = cnx1.cursor()
cursor2 = cnx2.cursor()

#Get data from MySQL
selQuery = ("SELECT E.stockExchange, E.symbol, E.company FROM stockExchange E ORDER BY E.stockExchangeId")

#Save Board of Directors data
insQuery1 = ("INSERT INTO companyBoard "
                "(memberName, memberTitle, company, stockSymbol, stockExchange) "
                "VALUES (%(name)s, %(title)s, %(company)s, %(symbol)s, %(exchange)s)")

#Save company executives data
insQuery2 = ("INSERT INTO keyExecutive "
                "(executiveName, executiveTitle, company, stockSymbol, stockExchange) "
                "VALUES (%(name)s, %(title)s, %(company)s, %(symbol)s, %(exchange)s)")

cursor1.execute(selQuery)

for (stockExchange, symbol, company) in cursor1:
    if stockExchange == 'amex':
        exchange = 'XASE'
    elif stockExchange == 'nasdaq':
        exchange = 'XNAS'
    elif stockExchange == 'nyse':
        exchange = 'XNYS'
    
    if symbol.isalpha():
        time.sleep(5)
        #Call for board of directors
        #Create webpage address
        pageAddress = webpage.replace('exchange', exchange)
        pageAddress = pageAddress.replace('ticker', symbol)
        pageAddress = pageAddress.replace('callType', 'Director')

        #Send a request
        pageData = requests.get(pageAddress)

        #Format out
        pageOutput = BeautifulSoup(pageData.content, 'html.parser')
        directorName = pageOutput.findAll('h2')
        directorTitle = pageOutput.findAll('span')
        
        #Save each board member info in MySQL database
        for i in range(0, len(directorName)):
            dOut1 = (directorName[i].text).split(',')
            dName = dOut1[0]
            dOut2 = (directorTitle[i].text).split('Since')
            dTitle = dOut2[0]
            mData = {}
            mData['symbol'] = symbol
            mData['company'] = company
            mData['name'] = dName
            mData['title'] = dTitle
            mData['exchange'] = stockExchange
            cursor2.execute(insQuery1, mData)
 
        #Call for company executives
        time.sleep(5)

        #Create webpage address
        pageAddress = pageAddress.replace('Director', 'Executive')

        #Send a request
        pageData = requests.get(pageAddress)

        #Format out
        pageOutput = BeautifulSoup(pageData.content, 'html.parser')
        directorName = pageOutput.findAll('h2')
        directorTitle = pageOutput.findAll('span')
        
        #Save each executive info in MySQL database
        for i in range(0, len(directorName)):
            dOut1 = (directorName[i].text).split(',')
            dName = dOut1[0]
            dOut2 = (directorTitle[i].text).split('Since')
            dTitle = dOut2[0]
            mData = {}
            mData['symbol'] = symbol
            mData['company'] = company
            mData['name'] = dName
            mData['title'] = dTitle
            mData['exchange'] = stockExchange
            cursor2.execute(insQuery2, mData)
            
    #Commit inserts
    cnx2.commit()

#Finally close all connections
cursor2.close()
cnx2.close()

cursor1.close()
cnx1.close()
"""

#Extract Facebook Data

Extract Facebook data from MySQL and save it to CSV. Upload CSV to GitHub.

In [ ]:
#Extract Facebook related data and save it CSV file
#Upload file to GitHub

import pandas as pd
import mysql.connector
"""
def getData(boardMember, cnx, query):
    cursor = cnx.cursor()
    selQuery = query.replace('reqMember', boardMember)
    cursor.execute(selQuery)
    tempDf = pd.DataFrame(cursor.fetchall())
    if not tempDf.empty:
        tempDf.columns = cursor.column_names
    cursor.close()
    return tempDf

cnx = mysql.connector.connect(user='root', host='127.0.0.1', database='company')

query = ("SELECT A.*,B.* FROM "
            "(SELECT DISTINCT memberName AS srcMember, memberTitle AS srcMemberTitle, company AS srcCompany FROM companyBoard WHERE memberName = 'reqMember' "
            "UNION "
            "SELECT DISTINCT executiveName, executiveTitle, company FROM keyExecutive WHERE executiveName = 'reqMember' "
            ") A "
            "INNER JOIN "
            "(SELECT memberName AS dstMember, memberTitle AS dstMemberTitle, C.company AS dstCompany "
            "FROM companyBoard C "
            "JOIN "
            "(SELECT company FROM keyExecutive WHERE executiveName = 'reqMember' "
            "UNION "
            "SELECT company FROM companyBoard WHERE memberName = 'reqMember' "
            ") D ON C.company = D.company "
            "WHERE memberName != 'reqMember' "
            "UNION "
            "SELECT executiveName, executiveTitle, E.company "
            "FROM keyExecutive E "
            "JOIN "
            "(SELECT company FROM keyExecutive WHERE executiveName = 'reqMember' "
            "UNION "
            "SELECT company FROM companyBoard WHERE memberName = 'reqMember' "
            ") F ON E.company = F.company "
            "WHERE executiveName != 'reqMember' "
            ") B ON A.srcCompany = B.dstCompany; ")
            
executiveDf = pd.DataFrame()

#Get Facebook board of directors and their association with other members
for i in range(3):
   if i==0:
       uniqueMemberList = ['Mark Zuckerberg']
   elif i>0:
       uniqueMemberList = executiveDf.dstMember.unique()
       executiveDf = pd.DataFrame()
   for reqMember in uniqueMemberList:
       boeDf = getData(reqMember, cnx, query)
       if not boeDf.empty:
           frames = [executiveDf, boeDf]
           executiveDf = pd.concat(frames)
      
#Get companies each board member is associated with
query = ("SELECT DISTINCT srcCompany, dstCompany "
        "FROM "
        "(SELECT memberName AS srcMember, memberTitle AS srcMemberTitle, company AS srcCompany "
        "FROM companyBoard WHERE memberName = 'reqMember' "
        "UNION "
        "SELECT executiveName, executiveTitle, company "
        "FROM keyExecutive WHERE executiveName = 'reqMember') A "
        "CROSS JOIN "
        "(SELECT memberName AS dstMember, memberTitle AS dstMemberTitle, company AS dstCompany "
        "FROM companyBoard WHERE memberName = 'reqMember' "
        "UNION "
        "SELECT executiveName, executiveTitle, company "
        "FROM keyExecutive WHERE executiveName = 'reqMember') B "
        "WHERE A.srcCompany != B.dstCompany; ")

#Get List of people to extract company they belong to
uniqueMemberList = executiveDf.srcMember.unique()

companyDf = pd.DataFrame()

#Get List of people in complete network
for reqMember in uniqueMemberList:
    tmpCompanyDf = getData(reqMember, cnx, query)
    if not tmpCompanyDf.empty:
        frames = [companyDf, tmpCompanyDf]
        companyDf = pd.concat(frames)

cnx.close()

companyDf = companyDf.reset_index(drop=True)
companyDf = companyDf.drop_duplicates()

companyFinalDf = companyDf.query('srcCompany == "dummy"')

for index, row in companyDf.iterrows():
    s = row[0]
    d = row[1]
    e = False
    if ((companyFinalDf['srcCompany'] == s) & (companyFinalDf['dstCompany'] == d)).any():
        e = True
    if ((companyFinalDf['srcCompany'] == d) & (companyFinalDf['dstCompany'] == s)).any():
        e = True
    if not e:
        frames = [companyFinalDf, companyDf[(companyDf.srcCompany == s) & (companyDf.dstCompany == d)]]
        companyFinalDf = pd.concat(frames)


companyFinalDf.to_csv("D:/CUNY/620/Week01/company.csv", sep=',', index = False)
"""

#Data

Download CSV file from GitHub and load it into dataframe. Generate network diagram using *Graphlab Create*.

In [6]:
import pandas as pd
import graphlab
from IPython.display import display
from IPython.display import Image

#Set canvas to display in jupyter notebook
graphlab.canvas.set_target('ipynb')

url="https://raw.githubusercontent.com/akulapa/Akula-DATA620-Week01-Homework/master/company.csv"
companyFinalDf = pd.read_csv(url, sep=',', error_bad_lines=False)

uniqueCompanyList = pd.DataFrame(companyFinalDf.srcCompany.unique())
uniqueCompanyList.columns = ['company']

#Load vertices and edges
vsf = graphlab.SFrame(data = uniqueCompanyList)
esf = graphlab.SFrame(data = companyFinalDf)

#Complete Facebook network at second level
g = graphlab.SGraph(vertices=vsf, edges=esf, vid_field='company', src_field='srcCompany', dst_field='dstCompany')
g.show(vlabel='id', highlight=['Facebook Inc'], arrows=True)

In [16]:
#vertices
vsf

company
Morgan Stanley
Norfolk Southern
Facebook Inc
Netflix Inc
Ingredion Incorporated
Balchem Corporation
Walt Disney
General Motors Company
Hewlett Packard
Broadridge FinancialSolutions Inc ...


In [17]:
#edges
esf

srcCompany,dstCompany
Morgan Stanley,Facebook Inc
Norfolk Southern,Facebook Inc
Facebook Inc,Morgan Stanley
Norfolk Southern,Morgan Stanley
Facebook Inc,Norfolk Southern
Morgan Stanley,Norfolk Southern
Netflix Inc,Facebook Inc
Facebook Inc,Netflix Inc
Ingredion Incorporated,Balchem Corporation
Facebook Inc,Balchem Corporation


In [8]:
print g.summary()

{'num_edges': 276L, 'num_vertices': 86L}


#Relationship

Following graph show relationship between companies Facebook, Inc, Apple Inc, Nike, Inc and Microsoft Corporation. Companies are related by *Board of Directors*.

In [9]:
targets = ['Facebook Inc', 'Apple Inc', 'Nike Inc','Microsoft Corporation']
subgraph = g.get_neighborhood(ids=targets, radius=1, full_subgraph=True)
subgraph.show(vlabel='id', highlight=['Facebook Inc', 'Apple Inc', 'Nike Inc','Microsoft Corporation'], arrows=True)

#References
- http://www.morningstar.com/
- https://turi.com/learn/userguide/sgraph/sgraph.html